In [18]:
from __future__ import print_function
import pickle
import os
import subprocess
import requests
from pprint import pprint
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from Naked.toolshed.shell import execute_js
from git import Repo
import virtualenv
import pip

In [2]:
def findTestPath(repoPath):
    testPath = None
    return testPath

def findRequirements(repoPath):
    return

In [27]:
class CodeCov:
    def __init__(self):
        self.repoName = None
        self.repoPath = '/home/mathius/Documents/CS5850/DataRetrieval/RepoDir/'
        self.repoUrl = None
        self.gitToken = None

        self.googleService = None
        self.tempEnvDir = os.path.join(os.path.expanduser("~"), ".codecov")

    def setup(self):
        f = open('creds/github-token.txt','r')
        self.gitToken = f.read()

        self.buildSheetsService()
        self.repoName = 'dateutil'
        self.repoPath += self.repoName
        
        # create a virtualenv at ~/.pypi_autoload
        if not os.path.exists(self.tempEnvDir):
            virtualenv.create_environment(self.tempEnvDir)
        activate_script = os.path.join(self.tempEnvDir, "bin", "activate_this.py")
        execfile(activate_script, dict(__file__=activate_script))

    def retrieveRepo(self):
        url = 'https://api.github.com/search/repositories?q=' + self.repoName
        headers= {'Authorization': 'token ' + self.gitToken}
        r = requests.get(url, headers=headers)
        if r.status_code != 200:
            raise Exception('Failed while getting repo data from Github API')
        self.repoUrl = r.json()['items'][0]['full_name']
        Repo.clone_from('https://github.com/' + self.repoUrl, self.repoPath)

    def setupRepo(self):
        reqFilePaths = findRequirements(self.repoPath)
        for filePath in reqFilePaths:
            with open(filePath) as f:
                for line in f:
                    # call pip's main function with each requirement
                    pip.main(['install','-U', line])
        
    def runTests(self):
        test_path = findTestPath(repoPath)

        if test_path == None:
            raise Exception('Failed to find test directory')

        cmd = ['coverage', 'run', '-m', 'pytest', test_path]
        proc = subprocess.Popen(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        o, e = proc.communicate()

        # print('Output: ' + o.decode('ascii'))
        # print('Error: '  + e.decode('ascii'))
        # print('code: ' + str(proc.returncode))

        # Failed to find pytest tests
        if proc.returncode != 0:
            cmd = ['coverage', 'run', '-m', 'unittest', 'discover', '-s', test_path]
            proc = subprocess.Popen(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
            o, e = proc.communicate()
            if proc.returncode != 0:
                raise Exception("Failed to run tests")

        cmd = ['coverage', 'xml']
        proc = subprocess.Popen(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        o, e = proc.communicate()

        if proc.returncode != 0:
            raise Exception("Failed to create coverage report")

    def scrape(self):
        success = execute_js('scrape.js', arguments='nelsonic/github-scraper')
        if not success:
            raise Exception("Failed while scraping Github")

    def organizeData(self):
        return

    def buildSheetsService(self):
        # If modifying these scopes, delete the file token.pickle.
        SCOPES = ['https://www.googleapis.com/auth/spreadsheets']

        # Build Google sheets service
        creds = None
        # The file token.pickle stores the user's access and refresh tokens, and is
        # created automatically when the authorization flow completes for the first
        # time.
        if os.path.exists('creds/token.pickle'):
            with open('creds/token.pickle', 'rb') as token:
                creds = pickle.load(token)
        # If there are no (valid) credentials available, let the user log in.
        if not creds or not creds.valid:
            if creds and creds.expired and creds.refresh_token:
                creds.refresh(Request())
            else:
                flow = InstalledAppFlow.from_client_secrets_file(
                    'creds/credentials.json', SCOPES)
                creds = flow.run_local_server()
            # Save the credentials for the next run
            with open('creds/token.pickle', 'wb') as token:
                pickle.dump(creds, token)

        self.googleService = build('sheets', 'v4', credentials=creds)

    def addToSheets(self, values):
        # The ID and range of a sample spreadsheet.
        spreadsheet_id = '1nhttU75rL_9gRf-Awr7YojMuPTIVyrDMxpHfaThJm6E'
        range_ = 'Sheet1!A1:E1'
        value_input_option = 'RAW'
        insert_data_option = 'INSERT_ROWS'

    #     values = [
    #         ['something','something']
    #     ]
        value_range_body = {
            'values': values
        }

        request = self.googleService.spreadsheets().values().append(
                        spreadsheetId=spreadsheet_id, 
                        range=range_, 
                        valueInputOption=value_input_option, 
                        insertDataOption=insert_data_option, 
                        body=value_range_body)

        response = request.execute()
        return response
    
    def cleanUp(self):
        os.system('deactivate')
        try:
            shutil.rmtree(self.tempEnvDir)
            shutil.rmtree(self.repoPath)
        except OSError as e:
            print ("Error: %s - %s." % (e.filename, e.strerror))
            raise e

In [28]:
codeCov = CodeCov()
codeCov.setup()
codeCov.retrieveRepo()

In [14]:
print(result)

{'spreadsheetId': '1nhttU75rL_9gRf-Awr7YojMuPTIVyrDMxpHfaThJm6E', 'tableRange': 'Sheet1!A1:B3', 'updates': {'spreadsheetId': '1nhttU75rL_9gRf-Awr7YojMuPTIVyrDMxpHfaThJm6E', 'updatedRange': 'Sheet1!A4:B4', 'updatedRows': 1, 'updatedColumns': 2, 'updatedCells': 2}}
